In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
data = pd.read_csv('cleaned_data.csv')
data.head(10)

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
1,1,You Won’t Get What You Want,Daughters,you know how life goes: one day you’re making ...,Ipecac,8.0,November,9,2018,Rock,NaN,NaN,NaN
2,2,Wham! Bang! Pow! Let’s Rock Out!,Art Brut,"on ,’s 2005 debut, ,, eddie argos made gaming ...",Alcopop!,7.0,November,24,2018,Rock,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
4,4,Temporary etc.,Booker Stardrum,it might seem counterintuitive for a drummer t...,NNA Tapes,7.7,December,20,2018,Experimental,NaN,NaN,NaN
5,6,Moondance,Van Morrison,"“will you shut the fuck up and listen?” , aske...",Warner Bros.,8.4,November,25,2018,Rock,NaN,NaN,NaN
6,7,Ghost Box (Expanded),SUSS,if you want to approximate the hypothetical es...,Northern Spy,7.6,November,26,2018,Experimental,NaN,NaN,NaN
7,8,Poison,Swizz Beatz,“it’s a difference between a beatmaker and a p...,Epic Records,7.1,November,9,2018,Rap,NaN,NaN,NaN
8,9,Lost Time Behind the Moon,Scott Hirsch,’s long musical journey has come full circle. ...,Scissor Tail,7.5,December,19,2018,Electronic,NaN,NaN,NaN
9,10,Acrylic,Leikeli47,"no one ever sees , unmasked. the omnipresent b...","Hardcover,RCA",7.6,November,26,2018,Rap,NaN,NaN,NaN


In [4]:
data18 = data.loc[data['Year']== '2018']
data18 = data18.loc[data18['Genre1']== 'Pop/R&B']
data18

,Unnamed: 0,album,artist,content,label,score,Month,Day,Year,Genre1,Genre2,Genre3,Genre4
0,0,El Mal Querer,Rosalía,"global bass, the internet-centric underground ...",Sony,8.8,November,8,2018,Pop/R&B,NaN,NaN,NaN
3,3,"4AM Mulholland EP,After 4AM EP","Amerie,Amerie",stung by file-sharing losses and the decline o...,"FEENIIX RISING ENTERTAINMENT, FEENIIX RISING ...","7.2,7.6",October,26,2018,Pop/R&B,NaN,NaN,NaN
75,76,The Dream My Bones Dream,Eiko Ishibashi,"memories of a distant place and time haunt ,, ...",Drag City,7.5,November,29,2018,Pop/R&B,NaN,NaN,NaN
91,92,Broken Politics,Neneh Cherry,"’s fifth album in 30 years is called ,, a refl...",Smalltown Supersound,7.3,October,24,2018,Pop/R&B,NaN,NaN,NaN
92,93,The Anteroom,How to Dress Well,how does one contend with the term “blue-eyed ...,Domino,7.3,October,24,2018,Pop/R&B,NaN,NaN,NaN
98,99,Honey,Robyn,’s work urges on our braver selves. without se...,"Konichiwa,Interscope",8.5,October,25,2018,Pop/R&B,NaN,NaN,NaN
108,109,The Pains of Growing,Alessia Cara,alessia cara was nearly drowned by a record co...,Def Jam,6.8,December,7,2018,Pop/R&B,NaN,NaN,NaN
127,128,LillyAnna,teddy<3,"in the mid-’00s, teddy geiger was a somewhat r...",Whatever,6.8,November,21,2018,Pop/R&B,NaN,NaN,NaN
129,130,Caution,Mariah Carey,"is synonymous with grandiosity, whether she’s...",Epic Records,7.5,November,22,2018,Pop/R&B,NaN,NaN,NaN
137,138,Shugga Sextape Vol. 1,Ian Isiah,"dolled-up brooklyn r&b singer , is a sex-posit...",UNO NYC,7.7,November,27,2018,Pop/R&B,NaN,NaN,NaN


In [5]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data18['content']))

print(data_words[:1])

[['global', 'bass', 'the', 'internet', 'centric', 'underground', 'scene', 'that', 'began', 'flourishing', 'around', 'the', 'world', 'in', 'the', 'late', 'emerged', 'when', 'young', 'producers', 'took', 'their', 'countries', 'folk', 'traditions', 'and', 'updated', 'them', 'for', 'millennial', 'dancefloor', 'they', 'used', 'new', 'technology', 'to', 'connect', 'previously', 'collapsed', 'musical', 'boundaries', 'cumbia', 'became', 'digitized', 'colombia', 'mexico', 'salsa', 'got', 'remixed', 'puerto', 'rico', 'new', 'york', 'canada', 'and', 'some', 'even', 'transmogrified', 'andean', 'pan', 'flutes', 'into', 'intellectual', 'bangers', 'global', 'bass', 'became', 'precursor', 'to', 'the', 'increasingly', 'boundless', 'way', 'pop', 'music', 'works', 'now', 'even', 'as', 'it', 'became', 'flattened', 'into', 'limp', 'and', 'canned', 'versions', 'of', 'what', 'some', 'people', 'dreadfully', 'termed', 'trop', 'house', 'not', 'to', 'mention', 'the', 'overall', 'poisonous', 'creep', 'of', 'edm',

In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['global', 'bass', 'the', 'internet', 'centric', 'underground', 'scene', 'that', 'began', 'flourishing', 'around', 'the', 'world', 'in', 'the', 'late', 'emerged', 'when', 'young', 'producers', 'took', 'their', 'countries', 'folk', 'traditions', 'and', 'updated', 'them', 'for', 'millennial', 'dancefloor', 'they', 'used', 'new', 'technology', 'to', 'connect', 'previously', 'collapsed', 'musical', 'boundaries', 'cumbia', 'became', 'digitized', 'colombia', 'mexico', 'salsa', 'got', 'remixed', 'puerto', 'rico', 'new_york', 'canada', 'and', 'some', 'even', 'transmogrified', 'andean', 'pan', 'flutes', 'into', 'intellectual', 'bangers', 'global', 'bass', 'became', 'precursor', 'to', 'the', 'increasingly', 'boundless', 'way', 'pop', 'music', 'works', 'now', 'even', 'as', 'it', 'became', 'flattened', 'into', 'limp', 'and', 'canned', 'versions', 'of', 'what', 'some', 'people', 'dreadfully', 'termed', 'trop', 'house', 'not', 'to', 'mention', 'the', 'overall', 'poisonous', 'creep', 'of', 'edm', 'in

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_lg')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['global', 'bass', 'internet', 'centric', 'underground', 'scene', 'begin', 'flourish', 'world', 'late', 'emerge', 'young', 'producer', 'take', 'country', 'folk', 'tradition', 'update', 'millennial', 'dancefloor', 'use', 'new', 'technology', 'connect', 'previously', 'collapse', 'musical', 'boundary', 'cumbia', 'become', 'digitize', 'mexico', 'salsa', 'get', 'remix', 'puerto', 'rico', 'new_york', 'canada', 'even', 'transmogrify', 'andean', 'pan', 'flute', 'intellectual', 'banger', 'global', 'bass', 'become', 'precursor', 'increasingly', 'boundless', 'way', 'pop', 'music', 'work', 'even', 'become', 'flatten', 'limp', 'canned', 'version', 'people', 'dreadfully', 'term', 'house', 'mention', 'overall', 'poisonous', 'creep', 'edm', 'bloodstream', 'concept', 'still', 'intriguing', 'internet', 'homogenize', 'individual', 'music', 'culture', 'big', 'studio', 'quantize', 'mish', 'mash', 'musician', 'retain', 'singularity', 'hyper', 'local', 'often', 'dissipate', 'history', 'culture', 'alternatel

In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

#filter extreme values in dictionary
id2word.filter_extremes(no_below=15, no_above=0.2, keep_n=50000)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 2), (23, 1), (24, 1), (25, 1), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 3), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1)]]


In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [11]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.048*"cry" + 0.037*"cool" + 0.035*"electronic" + 0.025*"duo" + '
  '0.024*"body" + 0.021*"pair" + 0.018*"convey" + 0.018*"club" + 0.018*"sign" '
  '+ 0.018*"opener"'),
 (1,
  '0.053*"political" + 0.025*"aesthetic" + 0.024*"appear" + 0.023*"stage" + '
  '0.023*"sample" + 0.021*"scene" + 0.020*"rock" + 0.020*"figure" + '
  '0.017*"character" + 0.017*"era"'),
 (2,
  '0.014*"bridge" + 0.014*"draw" + 0.013*"title_track" + 0.013*"collaborator" '
  '+ 0.012*"past" + 0.012*"lack" + 0.011*"define" + 0.010*"emotion" + '
  '0.010*"pure" + 0.010*"recall"'),
 (3,
  '0.036*"sex" + 0.024*"string" + 0.022*"small" + 0.021*"club" + '
  '0.018*"memory" + 0.018*"rest" + 0.017*"friend" + 0.016*"allow" + '
  '0.016*"rock" + 0.014*"human"'),
 (4,
  '0.023*"boy" + 0.020*"flow" + 0.017*"sample" + 0.017*"chord" + 0.017*"sort" '
  '+ 0.016*"club" + 0.016*"summer" + 0.015*"texture" + 0.014*"stretch" + '
  '0.014*"fan"'),
 (5,
  '0.021*"jazz" + 0.021*"collaborator" + 0.021*"closer" + 0.021*"american" + '


In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.244607353102658

Coherence Score:  0.21460549968075485


In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/davidbrostowitz/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.035679 -0.030770       1        1  21.050240
2     -0.016341  0.043771       2        1  18.826241
9     -0.055963 -0.014457       3        1  16.921059
3      0.077640 -0.087814       4        1   8.983433
8     -0.010959 -0.022046       5        1   8.927048
6      0.186157 -0.142877       6        1   8.790378
4      0.105936  0.202281       7        1   7.872487
0     -0.031861  0.115047       8        1   4.989192
1     -0.206009 -0.069416       9        1   3.157983
5     -0.012921  0.006281      10        1   0.481938, topic_info=    Category       Freq          Term      Total  loglift  logprob
285  Default  31.000000     political  31.000000  30.0000  30.0000
132  Default  34.000000  collaborator  34.000000  29.0000  29.0000
183  Default  35.000000           cry  35.000000  28.0000  28.0000
104  Default  30.000000          jazz  30.000000  27.0000  27.0000
155  Default  38.000000          cool  38.000000  26.0000  26.0000
286  Default  68.000000           sex  68.000000  25.0000  25.0000
52   Default  26.000000        closer  26.000000  24.0000  24.0000
97   Default  29.000000           era  29.000000  23.0000  23.0000
260  Default  23.000000      american  23.000000  22.0000  22.0000
200  Default  26.000000       promise  26.000000  21.0000  21.0000
176  Default  32.000000         white  32.000000  20.0000  20.0000
120  Default  49.000000          rock  49.000000  19.0000  19.0000
153  Default  44.000000          club  44.000000  18.0000  18.0000
114  Default  34.000000          past  34.000000  17.0000  17.0000
180  Default  32.000000          baby  32.000000  16.0000  16.0000
59   Default  23.000000    electronic  23.000000  15.0000  15.0000
325  Default  17.000000     statement  17.000000  14.0000  14.0000
35   Default  33.000000        sample  33.000000  13.0000  13.0000
219  Default  36.000000   arrangement  36.000000  12.0000  12.0000
363  Default  38.000000           duo  38.000000  11.0000  11.0000
152  Default  34.000000          body  34.000000  10.0000  10.0000
26   Default  22.000000     narrative  22.000000   9.0000   9.0000
171  Default  37.000000        rapper  37.000000   8.0000   8.0000
42   Default  38.000000       version  38.000000   7.0000   7.0000
175  Default  30.000000        vision  30.000000   6.0000   6.0000
343  Default  26.000000         drive  26.000000   5.0000   5.0000
7    Default  24.000000       concept  24.000000   4.0000   4.0000
2    Default  20.000000        appear  20.000000   3.0000   3.0000
109  Default  30.000000        memory  30.000000   2.0000   2.0000
89   Default  22.000000          vibe  22.000000   1.0000   1.0000
..       ...        ...           ...        ...      ...      ...
173  Topic10   0.482648     territory  12.923585   2.0476  -4.4937
52   Topic10   0.895587        closer  26.203728   1.9589  -3.8755
108  Topic10   0.482981          land  14.636882   1.9238  -4.4930
227  Topic10   0.482664      industry  14.840014   1.9093  -4.4936
86   Topic10   0.483392      surprise  15.398693   1.8739  -4.4921
231  Topic10   0.482788   perspective  15.489220   1.8668  -4.4934
104  Topic10   0.896026          jazz  30.641230   1.8030  -3.8750
274  Topic10   0.516836         ready  17.862942   1.7924  -4.4252
355  Topic10   0.482679         short  17.328314   1.7544  -4.4936
317  Topic10   0.482647      intimacy  17.361057   1.7524  -4.4937
273  Topic10   0.482663       quality  18.484228   1.6898  -4.4936
132  Topic10   0.895609  collaborator  34.965767   1.6705  -3.8754
29   Topic10   0.483430         prove  18.968521   1.6655  -4.4920
37   Topic10   0.482659         serve  19.615559   1.6303  -4.4936
190  Topic10   0.482685         exist  20.153086   1.6034  -4.4936
43   Topic10   0.483249      year_old  21.593542   1.5355  -4.4924
26   Topic10   0.486537     narrative  22.172123   1.5158  -4.4856
89   Topic10   0.482645   